In [ ]:
import numpy as np
import pandas as pd
import optuna
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, log_idf_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import train_test_split, ranking_metrics_at_k


In [2]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [3]:
lastfm_df = (
    pd.read_csv(
        "/home/coder/projects/rec-sys-research/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv",
        sep="\t",
        header=None,
        usecols=[0, 2, 3],
        names=['user_id', 'item_id', 'play_count'],
    )
    .loc[:, ['user_id', 'item_id', 'play_count']]
    .dropna()
    .rename(columns={'play_count': 'target'})
)
lastfm_df['user_id'].nunique(), lastfm_df['item_id'].nunique(), lastfm_df.shape[0]

(358868, 292363, 17535451)

In [4]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    lastfm_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (358868, 292363), Val Shape: (358868, 292363), Test Shape: (358868, 292363)


In [ ]:
results_folder = "results/lastfm_360k_als"
results_filename = "lastfm_360k_als_results.csv"

import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    algorithms = {
        "ALS_factors=10": lambda: AlternatingLeastSquares(factors=10, regularization=10, random_state=42),
        "ALS_factors=100": lambda: AlternatingLeastSquares(factors=100, regularization=10, random_state=42),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "log_idf",
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "log_idf":
            weighted = log_idf_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "log_idf":
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            params["p"] = trial.suggest_float("p", 0.1, 1.5)
        weighted_train = get_weighted_matrix(train_mat, params)

        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ranking_metrics_at_k(model, train_mat, val_mat, K=20, show_progress=False)['ndcg']

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "log_idf", "power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    metrics_at_10 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    metrics_at_20 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": metrics_at_10['ndcg'],
        "Test NDCG@20": metrics_at_20['ndcg'],
        "Test Precision@10": metrics_at_10['precision'],
        "Test Precision@20": metrics_at_20['precision'],
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [6]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [7]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 16:57:45,764] A new study created in memory with name: no-name-33f44e06-75a8-4852-91ab-ea42ac8987de


Running optimization for ALS_factors=10 with no_weighting...


[I 2026-02-08 16:57:56,170] Trial 0 finished with value: 0.098124324373431 and parameters: {}. Best is trial 0 with value: 0.098124324373431.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,no_weighting,1,0.098124,0.082853,0.105003,0.096782,0.150003,4.090059,{}


In [8]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 16:58:25,961] A new study created in memory with name: no-name-ee10513e-0c4d-4895-b707-1fb66151da9b


Running optimization for ALS_factors=10 with bm25...


[I 2026-02-08 16:59:05,706] Trial 2 finished with value: 0.08893543651100015 and parameters: {'bm25_k1': 572.2647428255859, 'bm25_b': 0.6360723841657323}. Best is trial 2 with value: 0.08893543651100015.
[I 2026-02-08 16:59:06,136] Trial 0 finished with value: 0.10740796941917705 and parameters: {'bm25_k1': 71.21261352669313, 'bm25_b': 0.08623064120986179}. Best is trial 0 with value: 0.10740796941917705.
[I 2026-02-08 16:59:06,282] Trial 1 finished with value: 0.09336199898625913 and parameters: {'bm25_k1': 310.30709757881124, 'bm25_b': 0.7225484231238086}. Best is trial 0 with value: 0.10740796941917705.
[I 2026-02-08 16:59:06,336] Trial 3 finished with value: 0.10914336163395247 and parameters: {'bm25_k1': 57.14101733047764, 'bm25_b': 0.16421475440177014}. Best is trial 3 with value: 0.10914336163395247.
[I 2026-02-08 16:59:45,316] Trial 4 finished with value: 0.08857935632850825 and parameters: {'bm25_k1': 596.5869349454124, 'bm25_b': 0.42855599869058736}. Best is trial 3 with valu

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,bm25,20,0.114727,0.101373,0.124704,0.113777,0.169893,4.168091,"{'bm25_k1': 14.7302080538714, 'bm25_b': 0.3555..."


In [9]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)


[I 2026-02-08 17:02:17,098] A new study created in memory with name: no-name-75080d60-95fc-498f-99dc-3467e0368efd


Running optimization for ALS_factors=10 with tfidf...


[I 2026-02-08 17:02:27,952] Trial 0 finished with value: 0.11463989313355573 and parameters: {}. Best is trial 0 with value: 0.11463989313355573.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,tfidf,1,0.11464,0.101103,0.124489,0.113529,0.169735,4.165138,{}


In [10]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:02:58,896] A new study created in memory with name: no-name-01ce0c97-607d-4bce-ad49-eb5f9fd276dd


Running optimization for ALS_factors=10 with log...


[I 2026-02-08 17:03:09,512] Trial 0 finished with value: 0.10185071122685145 and parameters: {}. Best is trial 0 with value: 0.10185071122685145.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,log,1,0.101851,0.090715,0.111051,0.100451,0.149473,4.14152,{}


In [11]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log_idf", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:03:39,792] A new study created in memory with name: no-name-9dbd11a0-b0e4-4465-bd8e-bfad869fdfbd


Running optimization for ALS_factors=10 with log_idf...


[I 2026-02-08 17:04:19,417] Trial 3 finished with value: 0.04027166576919839 and parameters: {'conf_alpha': 129.95362045441567}. Best is trial 3 with value: 0.04027166576919839.
[I 2026-02-08 17:04:19,761] Trial 2 finished with value: 0.05351582258530307 and parameters: {'conf_alpha': 82.64526863792554}. Best is trial 2 with value: 0.05351582258530307.
[I 2026-02-08 17:04:20,107] Trial 0 finished with value: 0.05330192991206934 and parameters: {'conf_alpha': 83.25631609580671}. Best is trial 2 with value: 0.05351582258530307.
[I 2026-02-08 17:04:20,164] Trial 1 finished with value: 0.0402078757592352 and parameters: {'conf_alpha': 130.27684191066538}. Best is trial 2 with value: 0.05351582258530307.
[I 2026-02-08 17:04:59,221] Trial 4 finished with value: 0.04438417492775081 and parameters: {'conf_alpha': 112.37257009934058}. Best is trial 2 with value: 0.05351582258530307.
[I 2026-02-08 17:04:59,635] Trial 7 finished with value: 0.05192212956245681 and parameters: {'conf_alpha': 87.07

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,log_idf,20,0.114382,0.101465,0.124241,0.113137,0.167997,4.133097,{'conf_alpha': 1.452506886543036}


In [12]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:07:29,905] A new study created in memory with name: no-name-5e2cc491-ad51-41fe-b028-434880a76c44


Running optimization for ALS_factors=10 with power...


[I 2026-02-08 17:08:08,961] Trial 3 finished with value: 0.10841698027375544 and parameters: {'power_p': 0.4816801985652096}. Best is trial 3 with value: 0.10841698027375544.
[I 2026-02-08 17:08:09,557] Trial 2 finished with value: 0.10979650192374027 and parameters: {'power_p': 0.5263500291754916}. Best is trial 2 with value: 0.10979650192374027.
[I 2026-02-08 17:08:09,669] Trial 0 finished with value: 0.10692721692490477 and parameters: {'power_p': 0.43976124824408047}. Best is trial 2 with value: 0.10979650192374027.
[I 2026-02-08 17:08:09,762] Trial 1 finished with value: 0.11224736118835603 and parameters: {'power_p': 0.66154541952071}. Best is trial 1 with value: 0.11224736118835603.
[I 2026-02-08 17:08:48,872] Trial 7 finished with value: 0.111157276673488 and parameters: {'power_p': 0.5819940100569424}. Best is trial 1 with value: 0.11224736118835603.
[I 2026-02-08 17:08:48,901] Trial 4 finished with value: 0.07864857231005215 and parameters: {'power_p': 1.1626296301802408}. Be

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,power,20,0.112377,0.099612,0.122228,0.111543,0.165945,4.21236,{'power_p': 0.695635863899009}


In [13]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:11:18,745] A new study created in memory with name: no-name-1419f7c1-4739-4b09-93ac-76b51ce6796b


Running optimization for ALS_factors=10 with normalized...


[I 2026-02-08 17:11:29,530] Trial 0 finished with value: 0.05065458415852462 and parameters: {}. Best is trial 0 with value: 0.05065458415852462.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,normalized,1,0.050655,0.043742,0.056652,0.051326,0.082385,4.144401,{}


In [14]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:12:00,058] A new study created in memory with name: no-name-344ef09c-6276-4183-b10e-30cd1ec256a2


Running optimization for ALS_factors=10 with pmi...


[I 2026-02-08 17:12:11,034] Trial 0 finished with value: 0.10314213648319413 and parameters: {}. Best is trial 0 with value: 0.10314213648319413.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,pmi,1,0.103142,0.09111,0.1118,0.101718,0.151538,4.12342,{}


In [15]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:12:41,922] A new study created in memory with name: no-name-77622f79-4aa4-4f34-9f3f-b9c3f5836a0a


Running optimization for ALS_factors=10 with robust_user_centric...


[I 2026-02-08 17:13:25,391] Trial 0 finished with value: 0.09949453585544063 and parameters: {'scale_factor': 6.26895678735921}. Best is trial 0 with value: 0.09949453585544063.
[I 2026-02-08 17:13:26,024] Trial 2 finished with value: 0.09773829622370225 and parameters: {'scale_factor': 5.033020200726049}. Best is trial 0 with value: 0.09949453585544063.
[I 2026-02-08 17:13:26,060] Trial 1 finished with value: 0.10103365158497832 and parameters: {'scale_factor': 7.670430513967003}. Best is trial 1 with value: 0.10103365158497832.
[I 2026-02-08 17:13:26,076] Trial 3 finished with value: 0.09739654529613115 and parameters: {'scale_factor': 4.833071407104602}. Best is trial 1 with value: 0.10103365158497832.
[I 2026-02-08 17:14:07,925] Trial 4 finished with value: 0.09980849226881341 and parameters: {'scale_factor': 6.512360801317806}. Best is trial 1 with value: 0.10103365158497832.
[I 2026-02-08 17:14:08,497] Trial 6 finished with value: 0.10234544297880736 and parameters: {'scale_facto

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,robust_user_centric,20,0.103062,0.092017,0.112377,0.101877,0.150936,4.148693,{'scale_factor': 9.973074394076782}


In [16]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:16:51,028] A new study created in memory with name: no-name-244b7a9c-4226-41cc-be5d-c3654ebcb9b7


Running optimization for ALS_factors=10 with robust_user_centric_weight_v2...


[I 2026-02-08 17:17:34,521] Trial 1 finished with value: 0.07664497542183366 and parameters: {'lower_q': 10.541360759458321, 'upper_q': 66.37232705098185}. Best is trial 1 with value: 0.07664497542183366.
[I 2026-02-08 17:17:34,921] Trial 2 finished with value: 0.07695482897372094 and parameters: {'lower_q': 18.68197701062833, 'upper_q': 58.75252667702381}. Best is trial 2 with value: 0.07695482897372094.
[I 2026-02-08 17:17:34,970] Trial 0 finished with value: 0.0763917846518961 and parameters: {'lower_q': 23.889987625774634, 'upper_q': 75.53960112490057}. Best is trial 2 with value: 0.07695482897372094.
[I 2026-02-08 17:17:35,028] Trial 3 finished with value: 0.07565321636723521 and parameters: {'lower_q': 37.905756168775724, 'upper_q': 92.15015711201204}. Best is trial 2 with value: 0.07695482897372094.
[I 2026-02-08 17:18:17,696] Trial 7 finished with value: 0.0763728838089095 and parameters: {'lower_q': 32.48262722508285, 'upper_q': 77.93103327883962}. Best is trial 2 with value: 

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,robust_user_centric_weight_v2,20,0.077028,0.07214,0.089279,0.081091,0.12238,4.126125,"{'lower_q': 19.745789021754597, 'upper_q': 55...."


In [17]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:21:01,695] A new study created in memory with name: no-name-98ee2c44-823b-4bf0-9421-449a2ba1891e


Running optimization for ALS_factors=10 with sigmoid_propensity...


[I 2026-02-08 17:21:41,074] Trial 1 finished with value: 0.1004780868225511 and parameters: {'p': 1.0361184073216585, 'beta': 0.9062561530952967}. Best is trial 1 with value: 0.1004780868225511.
[I 2026-02-08 17:21:41,271] Trial 3 finished with value: 0.09810400564078439 and parameters: {'p': 2.3387637597322577, 'beta': 0.6850155821705486}. Best is trial 1 with value: 0.1004780868225511.
[I 2026-02-08 17:21:41,410] Trial 2 finished with value: 0.09772089746895966 and parameters: {'p': 2.00369478459809, 'beta': 0.6130135861146923}. Best is trial 1 with value: 0.1004780868225511.
[I 2026-02-08 17:21:41,486] Trial 0 finished with value: 0.09559053924871333 and parameters: {'p': 0.15297058484745032, 'beta': 0.42616034857699014}. Best is trial 1 with value: 0.1004780868225511.
[I 2026-02-08 17:22:20,264] Trial 6 finished with value: 0.09235172586253858 and parameters: {'p': 1.3377602427644597, 'beta': 0.2005337183523318}. Best is trial 1 with value: 0.1004780868225511.
[I 2026-02-08 17:22:2

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,sigmoid_propensity,20,0.101058,0.089319,0.109792,0.099328,0.148732,4.12911,"{'p': 1.1341097073430944, 'beta': 0.9985206998..."


In [18]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:24:49,608] A new study created in memory with name: no-name-5e2e57fa-ae8a-46d0-9ef5-f01c11f53a86


Running optimization for ALS_factors=10 with power_lift...
Warning NaN Detected in row 8309 of 292363


[I 2026-02-08 17:25:28,927] Trial 3 finished with value: 0.05677357116353819 and parameters: {'p': 0.6924111512130507}. Best is trial 3 with value: 0.05677357116353819.
[I 2026-02-08 17:25:29,124] Trial 1 finished with value: 4.300474262583277e-05 and parameters: {'p': 1.324402930259383}. Best is trial 3 with value: 0.05677357116353819.
[I 2026-02-08 17:25:29,125] Trial 2 finished with value: 1.7682947835759303e-05 and parameters: {'p': 1.3623105495577228}. Best is trial 3 with value: 0.05677357116353819.
[I 2026-02-08 17:25:29,316] Trial 0 finished with value: 0.09035443603652571 and parameters: {'p': 0.39730398635939757}. Best is trial 0 with value: 0.09035443603652571.


Warning NaN Detected in row 171327 of 292363
Warning NaN Detected in row 8309 of 292363


[I 2026-02-08 17:26:08,184] Trial 4 finished with value: 0.09881950238226114 and parameters: {'p': 0.21339907316825835}. Best is trial 4 with value: 0.09881950238226114.
[I 2026-02-08 17:26:08,184] Trial 5 finished with value: 2.5075703760109938e-05 and parameters: {'p': 1.33955329048602}. Best is trial 4 with value: 0.09881950238226114.
[I 2026-02-08 17:26:08,531] Trial 7 finished with value: 0.06182400461674252 and parameters: {'p': 0.654295901502453}. Best is trial 4 with value: 0.09881950238226114.
[I 2026-02-08 17:26:08,616] Trial 6 finished with value: 0.055765637258635856 and parameters: {'p': 0.6989387937531315}. Best is trial 4 with value: 0.09881950238226114.
[I 2026-02-08 17:26:47,246] Trial 9 finished with value: 0.00021976473534892306 and parameters: {'p': 1.246056340747168}. Best is trial 4 with value: 0.09881950238226114.
[I 2026-02-08 17:26:47,312] Trial 8 finished with value: 0.09943525153336066 and parameters: {'p': 0.2510050045714566}. Best is trial 8 with value: 0.0

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=10,power_lift,20,0.099435,0.087465,0.10775,0.098083,0.14693,4.119206,{'p': 0.2510050045714566}


In [19]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:28:36,295] A new study created in memory with name: no-name-110d6ad4-75ac-4da7-8769-6563b74129ac


Running optimization for ALS_factors=100 with no_weighting...


[I 2026-02-08 17:28:55,684] Trial 0 finished with value: 0.1331736846153498 and parameters: {}. Best is trial 0 with value: 0.1331736846153498.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,no_weighting,1,0.133174,0.114862,0.145426,0.134103,0.207505,10.58023,{}


In [20]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:29:45,458] A new study created in memory with name: no-name-06174dc1-f541-4825-8e27-7484ba9fea0a


Running optimization for ALS_factors=100 with bm25...


[I 2026-02-08 17:31:01,514] Trial 3 finished with value: 0.10414566754925095 and parameters: {'bm25_k1': 657.5561511410366, 'bm25_b': 0.3541742118139899}. Best is trial 3 with value: 0.10414566754925095.
[I 2026-02-08 17:31:02,496] Trial 2 finished with value: 0.10055658279668893 and parameters: {'bm25_k1': 977.9897732881045, 'bm25_b': 0.5851591428419393}. Best is trial 3 with value: 0.10414566754925095.
[I 2026-02-08 17:31:02,549] Trial 1 finished with value: 0.10599690738358405 and parameters: {'bm25_k1': 408.95733498365263, 'bm25_b': 0.5322380707749446}. Best is trial 1 with value: 0.10599690738358405.
[I 2026-02-08 17:31:02,591] Trial 0 finished with value: 0.11250300307227683 and parameters: {'bm25_k1': 250.69225260673443, 'bm25_b': 0.13105462657899447}. Best is trial 0 with value: 0.11250300307227683.


Warning NaN Detected in row 256 of 292363


[I 2026-02-08 17:32:18,093] Trial 4 finished with value: 0.10252351803233697 and parameters: {'bm25_k1': 291.1787883525163, 'bm25_b': 0.9340335290665668}. Best is trial 0 with value: 0.11250300307227683.
[I 2026-02-08 17:32:18,489] Trial 6 finished with value: 0.10705923295172569 and parameters: {'bm25_k1': 488.70135887317446, 'bm25_b': 0.04683764243825517}. Best is trial 0 with value: 0.11250300307227683.
[I 2026-02-08 17:32:18,594] Trial 7 finished with value: 0.11132194646301106 and parameters: {'bm25_k1': 235.21638451449635, 'bm25_b': 0.528280161397747}. Best is trial 0 with value: 0.11250300307227683.
[I 2026-02-08 17:32:18,748] Trial 5 finished with value: 0.09443591350606045 and parameters: {'bm25_k1': 903.821290058812, 'bm25_b': 0.9247967975622147}. Best is trial 0 with value: 0.11250300307227683.
[I 2026-02-08 17:33:34,615] Trial 8 finished with value: 0.09791183819455268 and parameters: {'bm25_k1': 589.0179075945025, 'bm25_b': 0.8861925417768869}. Best is trial 0 with value: 

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,bm25,20,0.172291,0.161743,0.193159,0.175369,0.250875,10.662962,"{'bm25_k1': 2.0658641475236683, 'bm25_b': 0.32..."


In [21]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)


[I 2026-02-08 17:36:59,265] A new study created in memory with name: no-name-ace42c39-a454-4b15-b833-d7dbd8f40abf


Running optimization for ALS_factors=100 with tfidf...


[I 2026-02-08 17:37:19,256] Trial 0 finished with value: 0.16227168469924044 and parameters: {}. Best is trial 0 with value: 0.16227168469924044.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,tfidf,1,0.162272,0.147765,0.180146,0.165692,0.243394,10.624572,{}


In [22]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:38:09,878] A new study created in memory with name: no-name-bf198673-205d-4c0e-b483-3c3bca181236


Running optimization for ALS_factors=100 with log...


[I 2026-02-08 17:38:29,715] Trial 0 finished with value: 0.14671662129182217 and parameters: {}. Best is trial 0 with value: 0.14671662129182217.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,log,1,0.146717,0.139247,0.166679,0.151112,0.217007,10.584752,{}


In [23]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log_idf", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:39:20,055] A new study created in memory with name: no-name-1c8b06a4-a752-463a-8283-7e84aae38cc4


Running optimization for ALS_factors=100 with log_idf...


[I 2026-02-08 17:40:36,210] Trial 3 finished with value: 0.0708488043915151 and parameters: {'conf_alpha': 62.696101967960686}. Best is trial 3 with value: 0.0708488043915151.
[I 2026-02-08 17:40:36,697] Trial 2 finished with value: 0.06398712612987889 and parameters: {'conf_alpha': 89.49678588187838}. Best is trial 3 with value: 0.0708488043915151.
[I 2026-02-08 17:40:36,910] Trial 1 finished with value: 0.05842678614331893 and parameters: {'conf_alpha': 124.71349506272117}. Best is trial 3 with value: 0.0708488043915151.
[I 2026-02-08 17:40:36,944] Trial 0 finished with value: 0.07690683638755412 and parameters: {'conf_alpha': 47.81394956504833}. Best is trial 0 with value: 0.07690683638755412.


Warning NaN Detected in row 256 of 292363
Warning NaN Detected in row 32655 of 292363
Warning NaN Detected in row 292 of 292363


[I 2026-02-08 17:41:52,565] Trial 4 finished with value: 0.06102627301336009 and parameters: {'conf_alpha': 106.3594380554305}. Best is trial 0 with value: 0.07690683638755412.
[I 2026-02-08 17:41:52,738] Trial 7 finished with value: 0.05836622630054812 and parameters: {'conf_alpha': 125.22777604143234}. Best is trial 0 with value: 0.07690683638755412.
[I 2026-02-08 17:41:53,336] Trial 5 finished with value: 0.12410043096919228 and parameters: {'conf_alpha': 10.829289743452657}. Best is trial 5 with value: 0.12410043096919228.
[I 2026-02-08 17:41:53,353] Trial 6 finished with value: 0.07205486552940224 and parameters: {'conf_alpha': 59.17907376793566}. Best is trial 5 with value: 0.12410043096919228.


...

Warning NaN Detected in row 10497 of 292363


[I 2026-02-08 17:44:25,503] Trial 12 finished with value: 0.05704018724379895 and parameters: {'conf_alpha': 136.31119569481}. Best is trial 5 with value: 0.12410043096919228.
[I 2026-02-08 17:44:25,761] Trial 14 finished with value: 0.12280217289790683 and parameters: {'conf_alpha': 11.24395779694455}. Best is trial 5 with value: 0.12410043096919228.
[I 2026-02-08 17:44:25,875] Trial 13 finished with value: 0.1370652735693852 and parameters: {'conf_alpha': 7.400823853658778}. Best is trial 13 with value: 0.1370652735693852.
[I 2026-02-08 17:44:25,962] Trial 15 finished with value: 0.11797447282529186 and parameters: {'conf_alpha': 12.884475468326762}. Best is trial 13 with value: 0.1370652735693852.


Warning NaN Detected in row 41249 of 292363


[I 2026-02-08 17:45:41,904] Trial 18 finished with value: 0.1713809158509904 and parameters: {'conf_alpha': 1.1343218762096257}. Best is trial 18 with value: 0.1713809158509904.
[I 2026-02-08 17:45:42,402] Trial 16 finished with value: 0.12355671900120041 and parameters: {'conf_alpha': 11.012651433069841}. Best is trial 18 with value: 0.1713809158509904.
[I 2026-02-08 17:45:42,515] Trial 17 finished with value: 0.13090242998221044 and parameters: {'conf_alpha': 8.89552663683869}. Best is trial 18 with value: 0.1713809158509904.
[I 2026-02-08 17:45:42,578] Trial 19 finished with value: 0.08743470688265195 and parameters: {'conf_alpha': 32.27919993364166}. Best is trial 18 with value: 0.1713809158509904.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,log_idf,20,0.171381,0.159863,0.191756,0.174913,0.251558,10.577602,{'conf_alpha': 1.1343218762096257}


In [24]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:46:33,012] A new study created in memory with name: no-name-a04ca1ea-a93c-4f7c-b0b6-5d5c96308a9d


Running optimization for ALS_factors=100 with power...


[I 2026-02-08 17:47:48,979] Trial 3 finished with value: 0.14522646755342386 and parameters: {'power_p': 0.2928726638619408}. Best is trial 3 with value: 0.14522646755342386.
[I 2026-02-08 17:47:49,153] Trial 2 finished with value: 0.12288358297316573 and parameters: {'power_p': 0.12111445076990526}. Best is trial 3 with value: 0.14522646755342386.
[I 2026-02-08 17:47:49,382] Trial 1 finished with value: 0.11937605044457002 and parameters: {'power_p': 0.10038329738556864}. Best is trial 3 with value: 0.14522646755342386.
[I 2026-02-08 17:47:49,603] Trial 0 finished with value: 0.13633522563848188 and parameters: {'power_p': 0.21652915765933012}. Best is trial 3 with value: 0.14522646755342386.
[I 2026-02-08 17:49:04,915] Trial 4 finished with value: 0.15705758225216027 and parameters: {'power_p': 0.6926295218523403}. Best is trial 4 with value: 0.15705758225216027.
[I 2026-02-08 17:49:05,277] Trial 5 finished with value: 0.12249057950670991 and parameters: {'power_p': 0.118774132172963

Warning NaN Detected in row 292 of 292363
Warning NaN Detected in row 252 of 292363


[I 2026-02-08 17:51:36,508] Trial 12 finished with value: 0.12427899466315463 and parameters: {'power_p': 0.12992999364100793}. Best is trial 11 with value: 0.15714305785563276.
[I 2026-02-08 17:51:37,263] Trial 14 finished with value: 0.08203663401940979 and parameters: {'power_p': 1.3558278756994437}. Best is trial 11 with value: 0.15714305785563276.
[I 2026-02-08 17:51:37,325] Trial 13 finished with value: 0.07267859308098486 and parameters: {'power_p': 1.4102976390928754}. Best is trial 11 with value: 0.15714305785563276.
[I 2026-02-08 17:51:37,515] Trial 15 finished with value: 0.09964666105125501 and parameters: {'power_p': 1.2523513163189224}. Best is trial 11 with value: 0.15714305785563276.
[I 2026-02-08 17:52:52,503] Trial 17 finished with value: 0.1593196141872127 and parameters: {'power_p': 0.5510149323291363}. Best is trial 17 with value: 0.1593196141872127.
[I 2026-02-08 17:52:52,618] Trial 16 finished with value: 0.1000398583255322 and parameters: {'power_p': 1.249985715

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,power,20,0.159467,0.148507,0.178529,0.16326,0.235425,10.652491,{'power_p': 0.5700542103811785}


In [25]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:53:43,321] A new study created in memory with name: no-name-96b75647-70af-4817-a6d4-393f7b4e2d7f


Running optimization for ALS_factors=100 with normalized...


[I 2026-02-08 17:54:03,423] Trial 0 finished with value: 0.08277841357687131 and parameters: {}. Best is trial 0 with value: 0.08277841357687131.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,normalized,1,0.082778,0.072129,0.092692,0.084179,0.133654,10.585073,{}


In [26]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:54:53,863] A new study created in memory with name: no-name-aa2007c3-33b1-48c3-85a9-5710434271b4


Running optimization for ALS_factors=100 with pmi...


[I 2026-02-08 17:55:14,025] Trial 0 finished with value: 0.16532189995507757 and parameters: {}. Best is trial 0 with value: 0.16532189995507757.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,pmi,1,0.165322,0.155134,0.185279,0.167918,0.240383,10.564258,{}


In [27]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 17:56:04,628] A new study created in memory with name: no-name-7976f7e9-12a3-472b-b010-26a0c0efe8f1


Running optimization for ALS_factors=100 with robust_user_centric...


[I 2026-02-08 17:57:24,842] Trial 3 finished with value: 0.12567793044240075 and parameters: {'scale_factor': 3.1065516219751985}. Best is trial 3 with value: 0.12567793044240075.
[I 2026-02-08 17:57:25,023] Trial 1 finished with value: 0.13779019283496027 and parameters: {'scale_factor': 4.771753912401005}. Best is trial 1 with value: 0.13779019283496027.
[I 2026-02-08 17:57:25,055] Trial 0 finished with value: 0.12068881008070471 and parameters: {'scale_factor': 2.668265401986456}. Best is trial 1 with value: 0.13779019283496027.
[I 2026-02-08 17:57:25,191] Trial 2 finished with value: 0.1439845859624181 and parameters: {'scale_factor': 6.275846340054222}. Best is trial 2 with value: 0.1439845859624181.
[I 2026-02-08 17:58:44,557] Trial 4 finished with value: 0.12695063712115356 and parameters: {'scale_factor': 3.234367321333732}. Best is trial 2 with value: 0.1439845859624181.
[I 2026-02-08 17:58:45,089] Trial 6 finished with value: 0.09449153029336736 and parameters: {'scale_factor

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,robust_user_centric,20,0.152389,0.144301,0.172507,0.156579,0.224368,10.555187,{'scale_factor': 9.98038277416907}


In [28]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 18:03:37,751] A new study created in memory with name: no-name-ddb10e91-1076-43d0-95d9-c740404831e4


Running optimization for ALS_factors=100 with robust_user_centric_weight_v2...


[I 2026-02-08 18:04:57,812] Trial 1 finished with value: 0.08502207196959144 and parameters: {'lower_q': 19.861136671636906, 'upper_q': 91.67865883386678}. Best is trial 1 with value: 0.08502207196959144.
[I 2026-02-08 18:04:58,191] Trial 3 finished with value: 0.08613646635738026 and parameters: {'lower_q': 7.777844252292083, 'upper_q': 82.84429654061762}. Best is trial 3 with value: 0.08613646635738026.
[I 2026-02-08 18:04:58,317] Trial 0 finished with value: 0.08883450108098088 and parameters: {'lower_q': 30.133902549383368, 'upper_q': 64.48176166254207}. Best is trial 0 with value: 0.08883450108098088.
[I 2026-02-08 18:04:58,414] Trial 2 finished with value: 0.0870816489056866 and parameters: {'lower_q': 22.882762550210316, 'upper_q': 77.4550613498902}. Best is trial 0 with value: 0.08883450108098088.
/home/coder/projects/rec-sys-research/weighting_is_fun/weighting_strategies.py:364: RuntimeWarning: overflow encountered in exp
  weights = 1 / (1 + np.exp(-z_scores))
[I 2026-02-08 1

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,robust_user_centric_weight_v2,20,0.089661,0.08407,0.103182,0.093629,0.139709,10.619489,"{'lower_q': 44.649614431355324, 'upper_q': 57...."


In [29]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 18:11:10,307] A new study created in memory with name: no-name-08009416-fe03-4283-9174-04b538f50b6b


Running optimization for ALS_factors=100 with sigmoid_propensity...


[I 2026-02-08 18:12:26,254] Trial 0 finished with value: 0.13914108718442317 and parameters: {'p': 0.4832459716957699, 'beta': 0.5230445032064872}. Best is trial 0 with value: 0.13914108718442317.
[I 2026-02-08 18:12:26,603] Trial 2 finished with value: 0.14541725929494048 and parameters: {'p': 1.6157113916387627, 'beta': 0.6699230160951227}. Best is trial 2 with value: 0.14541725929494048.
[I 2026-02-08 18:12:26,899] Trial 1 finished with value: 0.149255643097996 and parameters: {'p': 2.8952721647927686, 'beta': 0.981716461173161}. Best is trial 1 with value: 0.149255643097996.
[I 2026-02-08 18:12:26,936] Trial 3 finished with value: 0.08999241587456772 and parameters: {'p': 3.8552398021291996, 'beta': 0.006481502990371868}. Best is trial 1 with value: 0.149255643097996.
[I 2026-02-08 18:13:42,711] Trial 7 finished with value: 0.13377512936880429 and parameters: {'p': 3.3111006521405932, 'beta': 0.4491166356081736}. Best is trial 1 with value: 0.149255643097996.
[I 2026-02-08 18:13:43

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,sigmoid_propensity,20,0.152111,0.144438,0.17262,0.156642,0.224425,10.595939,"{'p': 1.9503198056924926, 'beta': 0.9941970351..."


In [30]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 18:18:22,458] A new study created in memory with name: no-name-8c86c73e-9f85-47ce-b6fe-07acac014742


Running optimization for ALS_factors=100 with power_lift...
Warning NaN Detected in row 20781 of 292363
Warning NaN Detected in row 41249 of 292363


[I 2026-02-08 18:19:38,202] Trial 0 finished with value: 0.043692579515172504 and parameters: {'p': 1.1449211673463966}. Best is trial 0 with value: 0.043692579515172504.
[I 2026-02-08 18:19:38,733] Trial 1 finished with value: 0.11687487193918876 and parameters: {'p': 0.756897321313727}. Best is trial 1 with value: 0.11687487193918876.
[I 2026-02-08 18:19:38,884] Trial 2 finished with value: 0.1451098561065093 and parameters: {'p': 0.15902147699101196}. Best is trial 2 with value: 0.1451098561065093.
[I 2026-02-08 18:19:39,046] Trial 3 finished with value: 0.12960685756291587 and parameters: {'p': 0.6787702183783918}. Best is trial 2 with value: 0.1451098561065093.


Warning NaN Detected in row 20798 of 292363
Warning NaN Detected in row 289 of 292363
Warning NaN Detected in row 41204 of 292363
Warning NaN Detected in row 32053 of 292363
Warning NaN Detected in row 81671 of 292363
Warning NaN Detected in row 85 of 292363
Warning NaN Detected in row 904 of 292363
Warning NaN Detected in row 10494 of 292363
Warning NaN Detected in row 253 of 292363


[I 2026-02-08 18:20:54,020] Trial 4 finished with value: 0.033149946399417346 and parameters: {'p': 1.2171267392791472}. Best is trial 2 with value: 0.1451098561065093.
[I 2026-02-08 18:20:54,072] Trial 5 finished with value: 0.13651274888673812 and parameters: {'p': 0.12463175435456333}. Best is trial 2 with value: 0.1451098561065093.
[I 2026-02-08 18:20:54,777] Trial 7 finished with value: 0.13397081932646931 and parameters: {'p': 0.11531452837145646}. Best is trial 2 with value: 0.1451098561065093.
[I 2026-02-08 18:20:54,989] Trial 6 finished with value: 0.1518614079176649 and parameters: {'p': 0.4744039225752491}. Best is trial 6 with value: 0.1518614079176649.


Warning NaN Detected in row 72167 of 292363
Warning NaN Detected in row 281 of 292363
Warning NaN Detected in row 239 of 292363
Warning NaN Detected in row 281 of 292363


[I 2026-02-08 18:22:10,019] Trial 8 finished with value: 0.10521585048386944 and parameters: {'p': 0.8195042203909485}. Best is trial 6 with value: 0.1518614079176649.
[I 2026-02-08 18:22:10,387] Trial 10 finished with value: 0.08748809705229005 and parameters: {'p': 0.9108137386318094}. Best is trial 6 with value: 0.1518614079176649.
[I 2026-02-08 18:22:10,449] Trial 9 finished with value: 0.13408808456253388 and parameters: {'p': 0.11562094320933332}. Best is trial 6 with value: 0.1518614079176649.
[I 2026-02-08 18:22:10,616] Trial 11 finished with value: 0.1568227897190055 and parameters: {'p': 0.21114122944842095}. Best is trial 11 with value: 0.1568227897190055.
[I 2026-02-08 18:23:26,175] Trial 15 finished with value: 0.15191851054474337 and parameters: {'p': 0.4738493195029455}. Best is trial 11 with value: 0.1568227897190055.
[I 2026-02-08 18:23:26,185] Trial 12 finished with value: 0.16046806006163217 and parameters: {'p': 0.22886913616314505}. Best is trial 12 with value: 0.1

Warning NaN Detected in row 293 of 292363
Warning NaN Detected in row 270 of 292363
Warning NaN Detected in row 253 of 292363
Warning NaN Detected in row 20769 of 292363
Warning NaN Detected in row 31024 of 292363
Warning NaN Detected in row 41273 of 292363
Warning NaN Detected in row 22408 of 292363
Warning NaN Detected in row 167858 of 292363
Warning NaN Detected in row 4662 of 292363
Warning NaN Detected in row 5377 of 292363
Warning NaN Detected in row 160432 of 292363
Warning NaN Detected in row 110058 of 292363
Warning NaN Detected in row 101306 of 292363
Warning NaN Detected in row 133045 of 292363
Warning NaN Detected in row 253 of 292363
Warning NaN Detected in row 31017 of 292363
Warning NaN Detected in row 11386 of 292363
Warning NaN Detected in row 42621 of 292363
Warning NaN Detected in row 114311 of 292363
Warning NaN Detected in row 12367 of 292363
Warning NaN Detected in row 125731 of 292363
Warning NaN Detected in row 37707 of 292363
Warning NaN Detected in row 18387 o

[I 2026-02-08 18:24:42,396] Trial 18 finished with value: 0.16341999221643927 and parameters: {'p': 0.3351677931411075}. Best is trial 18 with value: 0.16341999221643927.
[I 2026-02-08 18:24:42,550] Trial 17 finished with value: 0.1595847956046291 and parameters: {'p': 0.38565889279165627}. Best is trial 18 with value: 0.16341999221643927.
[I 2026-02-08 18:24:42,801] Trial 19 finished with value: 0.16323888936282915 and parameters: {'p': 0.33851805566065335}. Best is trial 18 with value: 0.16341999221643927.
[I 2026-02-08 18:24:42,863] Trial 16 finished with value: 0.010887576157859478 and parameters: {'p': 1.4767825131376615}. Best is trial 18 with value: 0.16341999221643927.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,power_lift,20,0.16342,0.151488,0.181811,0.164628,0.237431,10.570601,{'p': 0.3351677931411075}


In [31]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,ALS_factors=100,bm25,20,0.172291,0.161743,0.193159,0.175369,0.250875,10.662962,"{'bm25_k1': 2.0658641475236683, 'bm25_b': 0.32..."
0,ALS_factors=100,log_idf,20,0.171381,0.159863,0.191756,0.174913,0.251558,10.577602,{'conf_alpha': 1.1343218762096257}
0,ALS_factors=100,pmi,1,0.165322,0.155134,0.185279,0.167918,0.240383,10.564258,{}
0,ALS_factors=100,power_lift,20,0.163420,0.151488,0.181811,0.164628,0.237431,10.570601,{'p': 0.3351677931411075}
0,ALS_factors=100,tfidf,1,0.162272,0.147765,0.180146,0.165692,0.243394,10.624572,{}
0,ALS_factors=100,power,20,0.159467,0.148507,0.178529,0.163260,0.235425,10.652491,{'power_p': 0.5700542103811785}
0,ALS_factors=100,sigmoid_propensity,20,0.152111,0.144438,0.172620,0.156642,0.224425,10.595939,"{'p': 1.9503198056924926, 'beta': 0.9941970351..."
0,ALS_factors=100,robust_user_centric,20,0.152389,0.144301,0.172507,0.156579,0.224368,10.555187,{'scale_factor': 9.98038277416907}
0,ALS_factors=100,log,1,0.146717,0.139247,0.166679,0.151112,0.217007,10.584752,{}
0,ALS_factors=100,no_weighting,1,0.133174,0.114862,0.145426,0.134103,0.207505,10.580230,{}
